In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import earthaccess
import h5netcdf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyinterp.backends.xarray  # Module that handles the filling of undefined values.
import pyinterp.fill
import seaborn as sns
import xarray as xr
import rioxarray as rxr
from matplotlib.patches import Rectangle
import logging
import xarray as xr
import dask.array as da
from dask_ml.decomposition import PCA
from dask_ml.cluster import KMeans
from dask.diagnostics import ProgressBar
from dask.distributed import Client

In [2]:
client = Client()

tspan = ("2024-04-01", "2025-04-30")
bbox = (123.084, -29.566, 138.024, -17.050)
res_rf = earthaccess.search_data(
    short_name="PACE_OCI_L3M_SFREFL",
    temporal=tspan,
    granule_name='*.MO.*0p1deg*',
    bounding_box=bbox
)
path_rf = earthaccess.open(res_rf)
ds_rf = xr.open_mfdataset(path_rf, combine="nested", concat_dim="date")
min_lon, max_lat, max_lon, min_lat = bbox
ds_rf = ds_rf.sel(lat=slice(min_lat, max_lat), lon=slice(min_lon, max_lon))
ds_rf

QUEUEING TASKS | :   0%|          | 0/13 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/13 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/13 [00:00<?, ?it/s]

<xarray.Dataset> Size: 119MB
Dimensions:     (date: 13, lat: 126, lon: 149, wavelength: 122, rgb: 3,
                 eightbitcolor: 256)
Coordinates:
  * wavelength  (wavelength) float64 976B 346.0 351.0 ... 2.131e+03 2.258e+03
  * lat         (lat) float32 504B -17.05 -17.15 -17.25 ... -29.35 -29.45 -29.55
  * lon         (lon) float32 596B 123.2 123.3 123.4 123.5 ... 137.8 137.9 138.0
Dimensions without coordinates: date, rgb, eightbitcolor
Data variables:
    rhos        (date, lat, lon, wavelength) float32 119MB dask.array<chunksize=(1, 2, 41, 8), meta=np.ndarray>
    palette     (date, rgb, eightbitcolor) uint8 10kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/62)
    product_name:                      PACE_OCI.20240401_20240430.L3m.MO.SFRE...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    cdm_data_type:                     grid
    identifier_product_doi_authority:  http://dx.doi.org
    identifier_product_doi:            10.5067/PACE/OCI/L3M/SFREFL/3.0
    data_bins:                         5359639
    data_minimum:                      -0.050000012
    data_maximum:                      1.0

In [3]:
dims_to_remove = {'rgb', 'eightbitcolor'}
vars_to_drop = [var for var in ds_rf.data_vars
                if dims_to_remove & set(ds_rf[var].dims)]
ds_rf = ds_rf.drop_vars(vars_to_drop)

In [ ]:



# ----------------------------------------
# Setup logging
# ----------------------------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = logging.getLogger()

# ----------------------------------------
# Begin preprocessing
# ----------------------------------------
log.info("Starting clustering pipeline...")

# Step 1: Chunk the dataset
log.info("Chunking dataset...")
chunked = ds_rf.chunk({'lat': 100, 'lon': 100})

# Step 2: Select variable and reshape
log.info("Stacking spatial dimensions and reshaping...")
stacked_da = chunked['rhos'].stack(samples=('lat', 'lon')).transpose('samples', 'date', 'wavelength')
X = stacked_da.data
X_flat = X.reshape((X.shape[0], -1))
log.info(f"Flattened data shape: {X_flat.shape}")

# Step 3: Rechunk
log.info("Rechunking to ensure one chunk along feature axis...")
X_flat = X_flat.rechunk({1: -1})

# Step 4: Fill NaNs
log.info("Filling NaNs with zeros...")
X_flat = da.where(da.isnan(X_flat), 0, X_flat)

# # Step 5: PCA
# log.info("Fitting PCA for dimensionality reduction...")
# pca = PCA(n_components=10)
# with ProgressBar():
#     X_reduced = pca.fit_transform(X_flat)
#     X_reduced.compute_chunk_sizes()
# log.info(f"PCA reduced shape: {X_reduced.shape}")
X_reduced = X_flat
# Step 6: KMeans
log.info("Fitting KMeans clustering...")
kmeans = KMeans(n_clusters=15, init_max_iter=1, max_iter=1, oversampling_factor=10, random_state=0)
with ProgressBar():
    kmeans.fit(X_reduced)
log.info("KMeans clustering complete.")

# Step 7: Convert labels back to spatial grid
log.info("Converting labels to xarray...")
labels = kmeans.labels_
labels_xr = xr.DataArray(labels, coords={'samples': stacked_da['samples']}, dims='samples')
labels_2d = labels_xr.unstack('samples')

log.info("Clustering pipeline complete.")

2025-08-05 18:05:20,551 - INFO - Starting clustering pipeline...
2025-08-05 18:05:20,552 - INFO - Chunking dataset...
2025-08-05 18:05:20,621 - INFO - Stacking spatial dimensions and reshaping...
2025-08-05 18:05:20,690 - INFO - Flattened data shape: (18774, 1586)
2025-08-05 18:05:20,691 - INFO - Rechunking to ensure one chunk along feature axis...
2025-08-05 18:05:20,696 - INFO - Filling NaNs with zeros...
2025-08-05 18:05:20,699 - INFO - Fitting KMeans clustering...
2025-08-05 18:05:20,701 - INFO - Starting _check_array
/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 11.49 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
